In [2]:
import sys
import os

# Get the absolute path of the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Add the project root directory to the Python path
sys.path.append(project_root)

In [3]:
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Define the path to the data directory
data_dir = os.path.join(project_root, 'data')

# Example: Access a specific data file in the data directory
data_file_path = os.path.join(data_dir, 'RADCURE_challenge_clinical.csv')
df = pd.read_csv(data_file_path)

In [4]:
from AutoML.analyzer import Analyzer

df.drop(columns=["Study ID", "split"], inplace=True)

Analyzer.dry_run(df)


Config file not found. Creating custom...
Used a heuristic to define categorical and continuous columns. Please review!

Categorical: ['T Stage', 'Stage', 'N Stage', 'Sex', 'Disease Site', 'Chemotherapy', 'Dose', 'HPV Combined', 'EGFRI', 'death']
Continuous: ['age at dx', 'survival_time']
  - Outliers found in T Stage: ['T2 (2): 1 out of 2552', 'TX: 1 out of 2552', 'T3 (2): 1 out of 2552']
╒══════════════════════════╤═══════════════════╤═══════════╤═════════════╕
│                          │                   │ Missing   │ Overall     │
╞══════════════════════════╪═══════════════════╪═══════════╪═════════════╡
│ n                        │                   │           │ 2552        │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ age at dx, mean (SD)     │                   │ 0         │ 62.0 (11.7) │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ survival_time, mean (SD) │                   │ 0         │ 4.8 (2.8)   │
├──────

{'columns': {'categorical': ['T Stage',
   'Stage',
   'N Stage',
   'Sex',
   'Disease Site',
   'Chemotherapy',
   'Dose',
   'HPV Combined',
   'EGFRI',
   'death'],
  'continuous': ['age at dx', 'survival_time'],
  'date': [],
  'other': []},
 'mapping': {'T Stage': {'T1': 'T1',
   'T3': 'T3',
   'T2': 'T2',
   'T4a': 'T4a',
   'T1a': 'T1a',
   'T4b': 'T4b',
   'Tis': 'Tis',
   'T4': 'T4',
   'T1b': 'T1b',
   'T2 (2)': 'Other',
   'TX': 'Other',
   'T3 (2)': 'Other'}},
 'missingness_strategy': {'categorical': {'T Stage': 'Unknown',
   'Stage': 'Unknown',
   'N Stage': 'Unknown',
   'Sex': 'Unknown',
   'Disease Site': 'Unknown',
   'Chemotherapy': 'Unknown',
   'Dose': 'Unknown',
   'HPV Combined': 'Unknown',
   'EGFRI': 'Unknown',
   'death': 'Unknown'},
  'continuous': {'age at dx': 'median', 'survival_time': 'median'}}}

In [5]:
from AutoML.analyzer import Analyzer

analyzer = Analyzer(df, target_variable='death', output_dir='./radcure_outputs/analyzer', one_hot_encode=True)

analyzer.run()

Config file not found. Creating custom...
Used a heuristic to define categorical and continuous columns. Please review!

Categorical: ['T Stage', 'Stage', 'N Stage', 'Sex', 'Disease Site', 'Chemotherapy', 'Dose', 'HPV Combined', 'EGFRI', 'death']
Continuous: ['age at dx', 'survival_time']
  - Outliers found in T Stage: ['T2 (2): 1 out of 2552', 'TX: 1 out of 2552', 'T3 (2): 1 out of 2552']
Applying changes from config...

╒══════════════════════════╤═══════════════════╤═══════════╤═════════════╕
│                          │                   │ Missing   │ Overall     │
╞══════════════════════════╪═══════════════════╪═══════════╪═════════════╡
│ n                        │                   │           │ 2552        │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ age at dx, mean (SD)     │                   │ 0         │ 62.0 (11.7) │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ survival_time, mean (SD) │                   │ 

In [6]:
from AutoML.trainer import TrainerSupervised

df = pd.read_csv('./radcure_outputs/analyzer/updated_data.csv', index_col=0)
df.rename(columns={'survival_time': 'time', 'death':'event'}, inplace=True)

trainer = TrainerSupervised(task='time_to_event', output_dir='./radcure_outputs/ED_trainer_explainer',)
trainer.run(df, ['event','time'])

Training CoxPH...
Training GradientBoosting...
Training RandomForest...
Training SVM...

C-index Scores:
CoxPH: 0.6473
GradientBoosting: 0.7760
RandomForest: 0.7679
SVM: 0.7746


In [7]:
from AutoML.explainer import Explainer

exp = Explainer.from_trainer(trainer)
exp.run()